In [1]:
import shap
import matplotlib.pyplot as plt

/home/g/10/week 8 9/Improved-detection-of-fraud-cases-for-e-commerce-and-bank-transactions/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import pickle
with open('model/random_forest_card.pkl', 'rb') as file:
    best_model = pickle.load(file)

UnpicklingError: STACK_GLOBAL requires str

In [6]:
import os
print(os.getcwd())  # Check current directory
print(os.listdir()) # List files in the current directory

/home/g/10/week 8 9/Improved-detection-of-fraud-cases-for-e-commerce-and-bank-transactions/notebooks
['model', 'Explainability.ipynb', 'data_analysis_and_preprocessing.ipynb', 'Model', 'Model_Building_and_Training.ipynb']


In [ ]:


# Select the best-performing model (assuming Random Forest for this example)
 # Replace with your best model based on evaluation

# Prepare the data for SHAP
# Use the training data for SHAP values calculation
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_fraud_train)

# Summary Plot
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_fraud_train, feature_names=X_fraud_train.columns)
plt.title("SHAP Summary Plot for Best Model (Random Forest)")
plt.show()

# Force Plot for a single prediction
# Choose an instance to explain (e.g., the first instance in the test set)
instance_to_explain = X_fraud_test.iloc[0]
shap_value_instance = explainer.shap_values(instance_to_explain)

plt.figure()
shap.force_plot(explainer.expected_value, shap_value_instance, instance_to_explain)
plt.title("SHAP Force Plot for a Single Prediction")
plt.show()